In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
# boston = load_boston()

# data = pd.DataFrame(boston.data)
# data.columns = boston.feature_names
# data['PRICE'] = boston.target

In [ ]:
# X, y = data.iloc[:,:-1],data.iloc[:,-1]
# data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
# xg_reg = xgb.XGBRegressor(objective ='reg:squarederror',
#                           colsample_bytree = 0.3,
#                           learning_rate = 0.1,
#                           max_depth = 5,
#                           alpha = 10,
#                           n_estimators = 10)
# xg_reg.fit(X_train, y_train)
# preds = xg_reg.predict(X_test)

In [ ]:
# rmse = np.sqrt(mean_squared_error(y_test, preds))
# print("RMSE: %f" % (rmse))

In [ ]:
import logging
import pathlib
import pickle
import warnings

import attr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
import xgboost as xgb
from joblib import Parallel, delayed
from sklearn.metrics import auc, precision_score, recall_score, roc_curve
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.neighbors import NearestNeighbors

from skrough.base import Bireduct
from skrough.bireducts.dynamically_adapted_approximate_bireduct import (
    DynamicallyAdaptedApproximateBireduct,
)

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

import config

In [ ]:
DATA_DIR = pathlib.Path(config.DATA_DIR)
TMP_DIR = pathlib.Path(config.TMP_DIR)
SEP = ";"


# FILEPATH_IN = DATA_DIR / 'toolbox_tabular_data_annonymized_discretized_uniform.csv'
# FILEPATH_IN = DATA_DIR / 'toolbox_tabular_data_annonymized_discretized_quantile.csv'
FILEPATH_IN = DATA_DIR / "toolbox_tabular_data_annonymized_discretized_kmeans.csv"
EMBEDDINGS_FILEPATH_IN = DATA_DIR / "process_embeddings_d50.csv"

In [ ]:
df = pd.read_csv(FILEPATH_IN, sep=SEP)
process_ids = df.pop("process_ids")
df = df.astype("category")
df = df.apply(lambda x: x.cat.codes)
df_dec = df.pop("target")


embeddings = pd.read_csv(EMBEDDINGS_FILEPATH_IN)
process_category = embeddings[["process_ids", "process_category"]]
embeddings.drop(["process_category"], axis=1, inplace=True)
embeddings.set_index("process_ids", inplace=True)
emb_nbrs = NearestNeighbors().fit(embeddings)

In [ ]:
from sklearn.utils import check_random_state

X_train, X_test, y_train, y_test = train_test_split(
    df, df_dec, test_size=0.8, random_state=123
)

cl = xgb.XGBClassifier(
    eval_metric="logloss",
    use_label_encoder=False,
    n_estimators=10,
    subsample=0.8,
    random_state=check_random_state(None),
)
cl.fit(X_train, y_train)
pred = cl.predict(X_test)

cl2 = xgb.XGBClassifier(
    eval_metric="logloss",
    use_label_encoder=False,
    n_estimators=10,
    subsample=0.8,
    random_state=check_random_state(None),
)
cl2.fit(X_train, y_train)
pred2 = cl2.predict(X_test)


print(accuracy_score(pred, y_test))
print(accuracy_score(pred2, y_test))

print(all(pred == pred2))